In [1]:
import sys
sys.path.append('../')

import os
import pandas as pd
import re
import numpy as np
from logparser import Spell, Drain
from tqdm import tqdm
#from logdeep.dataset.session import sliding_window
import glob
tqdm.pandas()
pd.options.mode.chained_assignment = None  # default='warn'


# In the first column of the log, "-" indicates non-alert messages while others are alert messages.
def count_anomaly(log_path):
    total_size = 0
    normal_size = 0
    with open(log_path, errors='ignore') as f:
        for line in f:
            total_size += 1
            if line.split('')[0] == '-':
                normal_size += 1
    print("total size {}, abnormal size {}".format(total_size, total_size - normal_size))


def deeplog_file_generator(filename, df, features):
    with open(filename, 'w') as f:
        for _, row in df.iterrows():
            for val in zip(*row[features]):
                f.write(','.join([str(v) for v in val]) + ' ')
            f.write('\n')




def sample_raw_data(data_file, output_file, sample_window_size, sample_step_size):
    # sample 1M by sliding window, abnormal rate is over 2%
    sample_data = []
    labels = []
    idx = 0

    # spirit dataset can start from the 2Mth line, as there are many abnormal lines gathering in the first 2M
    with open(data_file, 'r', errors='ignore') as f:
        for line in f:
            labels.append(line.split()[0] != '-')
            sample_data.append(line)

            if len(labels) == sample_window_size:
                abnormal_rate = sum(np.array(labels)) / len(labels)
                print(f"{idx + 1} lines, abnormal rate {abnormal_rate}")
                break

            idx += 1
            if idx % sample_step_size == 0:
                print(f"Process {round(idx/sample_window_size * 100,4)} % raw data", end='\r')

    with open(output_file, "w") as f:
        f.writelines(sample_data)

    print("Sampling done")




In [2]:
def parse_log(input_dir, output_dir, log_file, parser_type):
    #log_format = '<Label> <Id> <Date> <Admin> <Month> <Day> <Time> <AdminAddr> <Content>'
    log_format = '<Date> <Time>, <Type>             <Admin>  <Content>'
    regex = [
        r'(0x)[0-9a-fA-F]+',  # hexadecimal
        r'\d+\.\d+\.\d+\.\d+',
        r'(?<=Warning: we failed to resolve data source name )[\w\s]+',
        r'\d+'
    ]
    keep_para = False
    if parser_type == "drain":
        # the hyper parameter is set according to http://jmzhu.logpai.com/pub/pjhe_icws2017.pdf
        st = 0.3  # Similarity threshold
        depth = 3  # Depth of all leaf nodes

        # Drain is modified
        parser = Drain.LogParser(log_format,
                                 indir=input_dir,
                                 outdir=output_dir,
                                 depth=depth,
                                 st=st,
                                 rex=regex,
                                 keep_para=keep_para, maxChild=1000)
        parser.parse(log_file)

    elif parser_type == "spell":
        tau = 0.35
        parser = Spell.LogParser(indir=data_dir,
                                 outdir=output_dir,
                                 log_format=log_format,
                                 tau=tau,
                                 rex=regex,
                                 keep_para=keep_para)
        parser.parse(log_file)

In [ ]:
##File parser

data_dir = os.path.expanduser("../dataset/Windows")
output_dir = "../output/windows/"
raw_log_file = "xaa.txt"
sample_log_file = "xaa.txt"
sample_window_size = 2*10**7
sample_step_size = 10**4
window_name = ''
log_file = sample_log_file

parser_type = 'drain'
#mins
window_size = 1
step_size = 0.5
train_ratio = 6000

hresult_dict = {'0x00000000': 1,'0x800f080d':2,'0x800f0805':3, '0x80070490':4, '0x80004005':5, '0x80070001':6, '0x80071a2d':7, '0x80070216':8, '0x80070bc2':9, '0x800f0816':10, '0x800f0806':11, '0x800f0902':12, '0x80070002':13}

def matchfunc(line):
    # Capture one-or-more characters of non-whitespace after the initial match
    match = re.search(r'HRESULT = (\S+)', line)

    # Did we find a match?
    if match:
        # Yes, process it
        weather = match.group(1)
        return(hresult_dict[weather])
    return(0)

########
# count anomaly
########
# count_anomaly(data_dir + log_file)
# sys.exit()

import glob
file_list = glob.glob(data_dir +"/*.txt")
#########
# sample raw data
#########
#sample_raw_data(data_dir+raw_log_file, data_dir+sample_log_file, sample_window_size, sample_step_size )

a = glob.glob(data_dir +"/*.*") 
file_list=[os.path.basename(list_item) for list_item in a]
print(file_list)
##########
# Parser #
#########
for file in file_list:
    print("\nParsing", file)
    parse_log(data_dir, output_dir, file, parser_type)


##################
# Transformation #
##################
for file in file_list:
    print("\nTransforming", file)
    df = pd.read_csv(f'{output_dir}{file}_structured.csv')
    df['Label'] = 0

    #df.loc[df['Content'].str.contains("HRESULT"), "Label"] = 1
    df['Label'] = df.apply(lambda row: matchfunc(row['Content']), axis = 1)
    df.to_excel(f'{output_dir}{file}_labeled.xlsx')  
    



# csv files in the path
file_list = glob.glob(output_dir + "*_labeled.xlsx")
 
# list of excel files we want to merge.
# pd.read_excel(file_path) reads the excel
# data into pandas dataframe.
excl_list = []
 
for file in file_list:
    excl_list.append(pd.read_excel(file))
 
# create a new dataframe to store the
# merged excel file.
excl_merged = pd.DataFrame()
 
for excl_file in excl_list:
     
    # appends the data into the excl_merged
    # dataframe.
    excl_merged = excl_merged.append(
      excl_file, ignore_index=True)
 
# exports the dataframe into excel file with
# specified name.
excl_merged.to_csv('total_output.csv', index=False)
print("done!!")

['wrd00.txt', 'wrd01.txt', 'wrd02.txt', 'wrd03.txt', 'wrd04.txt', 'wrd05.txt', 'wrd06.txt', 'wrd07.txt', 'wrd08.txt', 'wrd09.txt', 'wrd10.txt', 'wrd11.txt', 'wrd12.txt', 'wrd13.txt', 'wrd14.txt', 'wrd15.txt', 'wrd16.txt', 'wrd17.txt', 'wrd18.txt', 'wrd19.txt', 'wrd20.txt', 'wrd21.txt', 'wrd22.txt', 'wrd23.txt', 'wrd24.txt', 'wrd25.txt', 'wrd26.txt', 'wrd27.txt', 'wrd28.txt', 'wrd29.txt', 'wrd30.txt', 'wrd31.txt', 'wrd32.txt', 'wrd33.txt', 'wrd34.txt', 'wrd35.txt', 'wrd36.txt', 'wrd37.txt', 'wrd38.txt', 'wrd39.txt', 'wrd40.txt', 'wrd41.txt', 'wrd42.txt', 'wrd43.txt', 'wrd44.txt', 'wrd45.txt', 'wrd46.txt', 'wrd47.txt', 'wrd48.txt', 'wrd49.txt', 'wrd50.txt', 'wrd51.txt', 'wrd52.txt', 'wrd53.txt', 'wrd54.txt', 'wrd55.txt', 'wrd56.txt', 'wrd57.txt', 'wrd58.txt', 'wrd59.txt', 'wrd60.txt', 'wrd61.txt', 'wrd62.txt', 'wrd63.txt', 'wrd64.txt', 'wrd65.txt', 'wrd66.txt', 'wrd67.txt', 'wrd68.txt', 'wrd69.txt', 'wrd70.txt', 'wrd71.txt', 'wrd72.txt', 'wrd73.txt', 'wrd74.txt', 'wrd75.txt', 'wrd76.txt'

Total size after encoding is 49966 50000
Parsing done. [Time taken: 0:00:04.744213]

Parsing wrd01.txt
Parsing file: ../dataset/Windows\wrd01.txt
Total size after encoding is 49996 50000
Parsing done. [Time taken: 0:00:04.703818]

Parsing wrd02.txt
Parsing file: ../dataset/Windows\wrd02.txt
Total size after encoding is 49992 50000
Parsing done. [Time taken: 0:00:05.284390]

Parsing wrd03.txt
Parsing file: ../dataset/Windows\wrd03.txt
Total size after encoding is 49996 50000
Parsing done. [Time taken: 0:00:05.114272]

Parsing wrd04.txt
Parsing file: ../dataset/Windows\wrd04.txt
Total size after encoding is 49997 50000
Parsing done. [Time taken: 0:00:05.123085]

Parsing wrd05.txt
Parsing file: ../dataset/Windows\wrd05.txt
Total size after encoding is 49972 50000
Parsing done. [Time taken: 0:00:04.605678]

Parsing wrd06.txt
Parsing file: ../dataset/Windows\wrd06.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:04.972535]

Parsing wrd07.txt
Parsing file: ../data

Parsing done. [Time taken: 0:00:05.038942]

Parsing wrd57.txt
Parsing file: ../dataset/Windows\wrd57.txt
Total size after encoding is 49769 50001
Parsing done. [Time taken: 0:00:05.451781]

Parsing wrd58.txt
Parsing file: ../dataset/Windows\wrd58.txt
Total size after encoding is 49996 50000
Parsing done. [Time taken: 0:00:05.413116]

Parsing wrd59.txt
Parsing file: ../dataset/Windows\wrd59.txt
Total size after encoding is 49991 50000
Parsing done. [Time taken: 0:00:05.437812]

Parsing wrd60.txt
Parsing file: ../dataset/Windows\wrd60.txt
Total size after encoding is 49997 50000
Parsing done. [Time taken: 0:00:05.582776]

Parsing wrd61.txt
Parsing file: ../dataset/Windows\wrd61.txt
Total size after encoding is 49997 50000
Parsing done. [Time taken: 0:00:05.104279]

Parsing wrd62.txt
Parsing file: ../dataset/Windows\wrd62.txt
Total size after encoding is 49972 50000
Parsing done. [Time taken: 0:00:04.768561]

Parsing wrd63.txt
Parsing file: ../dataset/Windows\wrd63.txt
Total size after en

Total size after encoding is 49990 50000
Parsing done. [Time taken: 0:00:04.831195]

Parsing wrd9023.txt
Parsing file: ../dataset/Windows\wrd9023.txt
Total size after encoding is 49990 50000
Parsing done. [Time taken: 0:00:04.461616]

Parsing wrd9024.txt
Parsing file: ../dataset/Windows\wrd9024.txt
Total size after encoding is 49986 50000
Parsing done. [Time taken: 0:00:04.699507]

Parsing wrd9025.txt
Parsing file: ../dataset/Windows\wrd9025.txt
Total size after encoding is 49433 50000
Parsing done. [Time taken: 0:00:04.740033]

Parsing wrd9026.txt
Parsing file: ../dataset/Windows\wrd9026.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:04.457858]

Parsing wrd9027.txt
Parsing file: ../dataset/Windows\wrd9027.txt
Total size after encoding is 49993 50000
Parsing done. [Time taken: 0:00:04.942032]

Parsing wrd9028.txt
Parsing file: ../dataset/Windows\wrd9028.txt
Total size after encoding is 49992 50000
Parsing done. [Time taken: 0:00:04.532752]

Parsing wrd9029

Total size after encoding is 49824 50000
Parsing done. [Time taken: 0:00:04.893371]

Parsing wrd9078.txt
Parsing file: ../dataset/Windows\wrd9078.txt
Total size after encoding is 49914 50000
Parsing done. [Time taken: 0:00:05.100779]

Parsing wrd9079.txt
Parsing file: ../dataset/Windows\wrd9079.txt
Total size after encoding is 49961 50000
Parsing done. [Time taken: 0:00:04.542570]

Parsing wrd9080.txt
Parsing file: ../dataset/Windows\wrd9080.txt
Total size after encoding is 49983 50000
Parsing done. [Time taken: 0:00:04.607986]

Parsing wrd9081.txt
Parsing file: ../dataset/Windows\wrd9081.txt
Total size after encoding is 49974 50000
Parsing done. [Time taken: 0:00:04.948330]

Parsing wrd9082.txt
Parsing file: ../dataset/Windows\wrd9082.txt
Total size after encoding is 49965 50000
Parsing done. [Time taken: 0:00:05.003887]

Parsing wrd9083.txt
Parsing file: ../dataset/Windows\wrd9083.txt
Total size after encoding is 49983 50000
Parsing done. [Time taken: 0:00:04.583667]

Parsing wrd9084

...

Parsing done. [Time taken: 0:00:05.760319]

Parsing wrd990798.txt
Parsing file: ../dataset/Windows\wrd990798.txt
Total size after encoding is 49729 50000
Parsing done. [Time taken: 0:00:05.143948]

Parsing wrd990799.txt
Parsing file: ../dataset/Windows\wrd990799.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:04.898351]

Parsing wrd990800.txt
Parsing file: ../dataset/Windows\wrd990800.txt
Total size after encoding is 49992 50000
Parsing done. [Time taken: 0:00:04.890269]

Parsing wrd990801.txt
Parsing file: ../dataset/Windows\wrd990801.txt
Total size after encoding is 49993 50000
Parsing done. [Time taken: 0:00:05.510044]

Parsing wrd990802.txt
Parsing file: ../dataset/Windows\wrd990802.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:05.833246]

Parsing wrd990803.txt
Parsing file: ../dataset/Windows\wrd990803.txt
Total size after encoding is 49988 50000
Parsing done. [Time taken: 0:00:05.374394]

Parsing wrd990804.txt
Parsing fi

Parsing done. [Time taken: 0:00:05.321885]

Parsing wrd990851.txt
Parsing file: ../dataset/Windows\wrd990851.txt
Total size after encoding is 49975 50000
Parsing done. [Time taken: 0:00:04.853291]

Parsing wrd990852.txt
Parsing file: ../dataset/Windows\wrd990852.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:05.086241]

Parsing wrd990853.txt
Parsing file: ../dataset/Windows\wrd990853.txt
Total size after encoding is 49994 50000
Parsing done. [Time taken: 0:00:04.974001]

Parsing wrd990854.txt
Parsing file: ../dataset/Windows\wrd990854.txt
Total size after encoding is 49977 50000
Parsing done. [Time taken: 0:00:05.199968]

Parsing wrd990855.txt
Parsing file: ../dataset/Windows\wrd990855.txt
Total size after encoding is 49978 50000
Parsing done. [Time taken: 0:00:05.273671]

Parsing wrd990856.txt
Parsing file: ../dataset/Windows\wrd990856.txt
Total size after encoding is 49989 50000
Parsing done. [Time taken: 0:00:05.129723]

Parsing wrd990857.txt
Parsing fi

Parsing done. [Time taken: 0:00:04.596998]

Parsing wrd990904.txt
Parsing file: ../dataset/Windows\wrd990904.txt
Total size after encoding is 49981 50000
Parsing done. [Time taken: 0:00:04.892352]

Parsing wrd990905.txt
Parsing file: ../dataset/Windows\wrd990905.txt
Total size after encoding is 49988 50000
Parsing done. [Time taken: 0:00:04.875658]

Parsing wrd990906.txt
Parsing file: ../dataset/Windows\wrd990906.txt
Total size after encoding is 49991 50000
Parsing done. [Time taken: 0:00:04.853336]

Parsing wrd990907.txt
Parsing file: ../dataset/Windows\wrd990907.txt
Total size after encoding is 49982 50000
Parsing done. [Time taken: 0:00:05.057469]

Parsing wrd990908.txt
Parsing file: ../dataset/Windows\wrd990908.txt
Total size after encoding is 49993 50000
Parsing done. [Time taken: 0:00:05.031531]

Parsing wrd990909.txt
Parsing file: ../dataset/Windows\wrd990909.txt
Total size after encoding is 49967 50000
Parsing done. [Time taken: 0:00:05.815487]

Parsing wrd990910.txt
Parsing fi

Parsing done. [Time taken: 0:00:04.599830]

Parsing wrd990957.txt
Parsing file: ../dataset/Windows\wrd990957.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:05.187400]

Parsing wrd990958.txt
Parsing file: ../dataset/Windows\wrd990958.txt
Total size after encoding is 49991 50000
Parsing done. [Time taken: 0:00:05.102539]

Parsing wrd990959.txt
Parsing file: ../dataset/Windows\wrd990959.txt
Total size after encoding is 49496 50237
Parsing done. [Time taken: 0:00:05.342623]

Parsing wrd990960.txt
Parsing file: ../dataset/Windows\wrd990960.txt
Total size after encoding is 49992 50000
Parsing done. [Time taken: 0:00:05.458838]

Parsing wrd990961.txt
Parsing file: ../dataset/Windows\wrd990961.txt
Total size after encoding is 49992 50000
Parsing done. [Time taken: 0:00:04.659842]

Parsing wrd990962.txt
Parsing file: ../dataset/Windows\wrd990962.txt
Total size after encoding is 50000 50000
Parsing done. [Time taken: 0:00:04.894298]

Parsing wrd990963.txt
Parsing fi

Parsing done. [Time taken: 0:00:05.740981]

Parsing wrd991010.txt
Parsing file: ../dataset/Windows\wrd991010.txt
Total size after encoding is 49990 50000
Parsing done. [Time taken: 0:00:06.947188]

Parsing wrd991011.txt
Parsing file: ../dataset/Windows\wrd991011.txt
Total size after encoding is 49978 50000
Parsing done. [Time taken: 0:00:06.436077]

Parsing wrd991012.txt
Parsing file: ../dataset/Windows\wrd991012.txt
Total size after encoding is 49983 50000
Parsing done. [Time taken: 0:00:05.569365]

Parsing wrd991013.txt
Parsing file: ../dataset/Windows\wrd991013.txt


In [ ]:
# sampling with sliding window
#deeplog_df = sliding_window(df[["timestamp", "Label", "EventId", "deltaT" <Date> <Time>, <Type>             <Admin>  <Content>]],
                            #para={"window_size": float(window_size)*60, "step_size": float(step_size) * 60}
                            #)
#output_dir += window_name

#########
# Train #
#########
df_normal = deeplog_df[deeplog_df["Label"] == 0]
df_normal = df_normal.sample(frac=1, random_state=12).reset_index(drop=True) #shuffle
normal_len = len(df_normal)
train_len = int(train_ratio) if train_ratio >= 1 else int(normal_len * train_ratio)

train = df_normal[:train_len]
deeplog_file_generator(os.path.join(output_dir,'train'), train, ["EventId"])
print("training size {}".format(train_len))


###############
# Test Normal #
###############
test_normal = df_normal[train_len:]
deeplog_file_generator(os.path.join(output_dir, 'test_normal'), test_normal, ["EventId"])
print("test normal size {}".format(normal_len - train_len))


#################
# Test Abnormal #
#################
df_abnormal = deeplog_df[deeplog_df["Label"] == 1]
deeplog_file_generator(os.path.join(output_dir,'test_abnormal'), df_abnormal, ["EventId"])
print('test abnormal size {}'.format(len(df_abnormal)))


In [ ]:
# specifying the path to csv files
path = "C:/downloads"
 
# csv files in the path
file_list = glob.glob(path + "/*.xlsx")
 
# list of excel files we want to merge.
# pd.read_excel(file_path) reads the excel
# data into pandas dataframe.
excl_list = []
 
for file in file_list:
    excl_list.append(pd.read_excel(file))
 
# create a new dataframe to store the
# merged excel file.
excl_merged = pd.DataFrame()
 
for excl_file in excl_list:
     
    # appends the data into the excl_merged
    # dataframe.
    excl_merged = excl_merged.append(
      excl_file, ignore_index=True)
 
# exports the dataframe into excel file with
# specified name.
excl_merged.to_excel('total_food_sales.xlsx', index=False)